In [1]:
from backends import izhikevich

In [2]:
from models import model_classes

In [3]:
import matnu

In [4]:
izhi = model_classes.IzhiModel()

JIT_IzhiBackend


In [5]:
izhi = model_classes.MATModel()

JIT_MATBackend
